<img src="logo.png" width="10%">

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import base64

from plotly import graph_objs as go
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, HTML
from IPython.display import IFrame
from IPython.display import display
from IPython.display import FileLink
from pyscal.crystal_structures import elements, structures
from pyscal_rdf.structure import StructureGraph

el_list = list(elements.keys())
st_list = list(structures.keys())

In [13]:
# Methods for visualisation

def create_box_plot(box, origin=[0,0,0]):
    """
    Create a plot of the simulation box
    """
    box = np.array(box)
    origin = np.array(origin)
    combos = list(itertools.combinations(range(3), 2))
    faces = []
    for combo in combos:
        f1 = [origin, box[combo[0]], box[combo[0]]+box[combo[1]], box[combo[1]], origin]
        s = combo[0] + combo[1]
        t = 3-s
        f2 = [origin + box[t], box[combo[0]]+ box[t],  box[combo[0]]+box[combo[1]]+ box[t], box[combo[1]]+ box[t], origin + box[t]]
        faces.append(np.array(f1))
        faces.append(np.array(f2))
    traces = []
    for face in faces:
        trace = go.Scatter3d(
            x=face[:,0],
            y=face[:,1],
            z=face[:,2],
            mode='lines',
            name='lines',
            line=dict(width=2.0, color='#263238'),
            showlegend=False
        )
        traces.append(trace)
    return traces

def plot_3d(sys):
    """
    Plot 3D of simulation box
    """
    pos = np.array(sys.atoms.positions)
    traces = create_box_plot(sys.box)
    radius = 10
    opacity = 1.0
    color = "#26c6da"
    data=go.Scatter3d(
        x=pos[:,0],
        y=pos[:,1],
        z=pos[:,2],
        mode='markers',
        opacity=1.0,
        marker=dict(
            sizemode='diameter',
            sizeref=750,
            size=radius,
            color = color,
            opacity = opacity,
            line=dict(width=0.5, color='#455A64')
        ),
    )
    traces.append(data)
    fig = go.Figure(data=traces)
    fig.update_layout(scene = dict(
                        xaxis_title="",
                        yaxis_title="",
                        zaxis_title="",
                        xaxis = dict(
                             showticklabels=False,
                             showbackground=False,
                             zerolinecolor="#455A64",),
                        yaxis = dict(
                            showticklabels=False,
                            showbackground=False,
                            zerolinecolor="#455A64"),
                        zaxis = dict(
                            showticklabels=False,
                            showbackground=False,
                            zerolinecolor="#455A64",),),
                        width=300,
                        height=300,
                        margin=dict(
                        r=10, l=10,
                        b=10, t=10)
                      )
    fig.update_layout(showlegend=False)
    fig.show()

In [14]:
#Element creation widgets
el_dropdown = widgets.Dropdown(
    options = el_list,
    value = "Fe",
    description = 'Element',
    disabled=False,
)

el_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

el_add_to_graph_check = widgets.Checkbox(
    value=True,
    description='Add to graph',
    disabled=False
)

el_repetition_box = widgets.IntText(
    value=1,
    description='Repetitions',
    disabled=False
)

el_out = widgets.Output(layout={'border': '1px solid black'})


def create_element(b=None):
    """
    Function to create element
    """
    element = el_dropdown.value
    add_to_graph = el_add_to_graph_check.value
    rep = el_repetition_box.value
    global sys
    sys = g.create_element(element, add_to_graph=add_to_graph, 
                          repetitions=[rep, rep, rep]) 
    el_out.clear_output()
    with el_out:
        plot_3d(sys)
    create_count_text()
    
@el_run_button.on_click
def element_plot_on_click(b):
    sys = create_element()

In [15]:
#Structure creation widgets
st_dropdown = widgets.Dropdown(
    options = st_list,
    value = "bcc",
    description = 'Structure',
    disabled=False,
)

st_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

st_add_to_graph_check = widgets.Checkbox(
    value=True,
    description='Add to graph',
    disabled=False
)

st_repetition_box = widgets.IntText(
    value=1,
    description='Repetitions',
    disabled=False
)

st_lattice_parameter_box = widgets.FloatText(
    value=1,
    description='Lattice Parameter',
    disabled=False
)

st_element_box = widgets.Text(
    value='Fe',
    description='Element',
    disabled=False
)

def create_structure(b=None):
    """
    Function to create structure
    """
    structure = st_dropdown.value
    add_to_graph = st_add_to_graph_check.value
    rep = st_repetition_box.value
    lattice_parameter = st_lattice_parameter_box.value
    element = st_element_box.value
    global sys
    sys = g.create_structure(structure, 
                             lattice_constant=lattice_parameter,
                             repetitions=[rep, rep, rep],
                             add_to_graph=add_to_graph,
                             element=element)
    
    el_out.clear_output()
    with el_out:
        plot_3d(sys)
    return sys
    
@st_run_button.on_click
def structure_plot_on_click(b):
    sys = create_structure()

In [16]:
read_formats = ['lammps-data', 'lammps-dump', 'poscar', 'cif']
write_formats = ['lammps-dump', 'lammps-data']

#format dropdown widget
read_format_dropdown = widgets.Dropdown(
    options = read_formats,
    value = "poscar",
    description = 'File format',
    disabled=False,
)

# possibility to upload file
file_upload = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

file_upload_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def read_structure(b=None):
    #write file
    with open("temp", "wb") as fout:
        fout.write(file_upload.value[0]['content'])
    global sys
    sys = g.read_structure("temp", format=read_format_dropdown.value)
    
    el_out.clear_output()
    with el_out:
        plot_3d(sys)
    return sys    
    
        
@file_upload_run_button.on_click
def process_file_on_click(b):
    sys = read_structure()
    
    
#items for download
write_format_dropdown = widgets.Dropdown(
    options = write_formats,
    value = "lammps-data",
    description = 'File format',
    disabled=False,
)

write_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

write_out = widgets.Output(layout={'border': '1px solid black'})

def write_structure(b=None):
    sys.to_file("outdata", format=write_format_dropdown.value)

@write_run_button.on_click
def write_file_on_click(b):
    write_structure()
    with open('outdata', 'r') as fin:
        res = fin.read()

    #FILE
    filename = 'res.txt'
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()

    #BUTTONS
    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info">Download File</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    write_out.clear_output()
    with write_out:
        display(HTML(html_button))

TraitError: Invalid selection: value not found

In [ ]:
#Arranging the widgets and tabbing them
text = "Create a lattice"
st_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>")      

text = "Create using element symbol"
el_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>")  
text = "Read structure"
read_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>")
text = "Download file"
write_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>")

el_panel_1 = VBox(children=[el_label,
                      el_dropdown,
                      el_add_to_graph_check,
                      el_repetition_box,
                      el_run_button,
                      st_label,
                      st_dropdown,
                      st_add_to_graph_check,
                      st_repetition_box,
                      st_lattice_parameter_box,
                      st_element_box,
                      st_run_button,
                      read_label,
                      HBox(children=[file_upload,
                                    read_format_dropdown,]),
                      file_upload_run_button])


text = "Visualise structure"
vis_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>") 
el_panel_2 = VBox(children=[ vis_label,
                      el_out,
                      write_label,
                      HBox(children=[write_format_dropdown,
                                    write_run_button]),
                      write_out,
                      #radius_slider,
                      #opacity_slider,
                      #color_box,
                      ])
element_tab = HBox(children=[el_panel_1, el_panel_2])

In [ ]:
#Widgets for querying

qt_query_box = widgets.Textarea(
    value="""
        PREFIX cmso: <https://purls.helmholtz-metadaten.de/cmso/>
        SELECT DISTINCT ?symbol
        WHERE {
            ?sample cmso:hasNumberOfAtoms ?number .
            ?sample cmso:hasMaterial ?material .
            ?material cmso:hasStructure ?structure .
            ?structure cmso:hasSpaceGroup ?spacegroup .
            ?spacegroup cmso:hasSpaceGroupSymbol ?symbol .
        FILTER (?number="2"^^xsd:integer)
}""",
    description='Element',
    disabled=False,
    layout=Layout(width="auto", height="100%")
)

out = widgets.Output(layout={'border': '1px solid black'})

qt_run_button = widgets.Button(
    description='Query',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

@qt_run_button.on_click
def query_plot_on_click(b):
    run_query()
    
def run_query(b=None):
    query = qt_query_box.value
    res = g.graph.query(query)
    out.clear_output()
    with out:
        for r in res:
            print(r[0])

query_tab = VBox(children=[qt_query_box,
                      qt_run_button,
                          out])


In [ ]:
#Ontology tab widgets
text = "pyscal-rdf uses the Computational Material Sample Ontology"
onto_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>") 

out_onto = widgets.Output(layout={'border': '1px solid black'})
onto_tab = VBox(children=[onto_label, out_onto])

with out_onto:
    display(IFrame(src="https://service.tib.eu/webvowl/#iri=http://purls.helmholtz-metadaten.de/cmso/", width='100%', height='500px'))

In [ ]:
#Create a structure graph
g = StructureGraph()

In [1]:
#final tab arrangement
tab = widgets.Tab(children=[element_tab, query_tab, onto_tab])
tab.set_title(0, 'Structure Creation')
tab.set_title(1, 'Run Query')
tab.set_title(2, 'Ontology')

count_out = widgets.Output(layout={'border': '1px solid black'})

def create_count_text():
    text = "Database has %d samples"%g.n_samples
    onto_label = widgets.HTML(value = f"<b><font color='#006064'>{text}</b>")
    count_out.clear_output()
    with count_out:
        display(onto_label)

count_update_button = widgets.Button(
    description='Update',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

@count_update_button.on_click
def count_update_on_click(b):
    create_count_text()
    
create_count_text()
main = widgets.VBox(children=[HBox(children=[count_out, count_update_button]), tab])
main

NameError: name 'widgets' is not defined

HTML(value='<html>\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n</head>\n<bo…